In [1]:
import numpy as np
from zeros import *

def OstarO(A, n=7):
    return (A.T @ A).round(n)

In [2]:
m = 5 # Number of measurements
k = 2 # Rank of O^*O

## Diagonalized prior 
prior = np.eye(m) * 100
prior[0,0] = 0.1
prior[1,1] = 0.2
prior[2,2] = 4

## The posterior precision at the first k eigenvectors 
precision = m/k + np.mean(np.diag(prior)[:k])


# M = O^*O
M = np.zeros((m,m))
for i in range(k):
    M[i,i] = precision - prior[i,i]

## Find O such that O^*O = M and O has unit norm rows
O = get_A(M, m).T
OstarO(O)

## Since O[2,:] = -O[4,:] we have clusterization

array([[2.55, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 2.45, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  ]])

# Clustered


In [59]:
m = 5
k = 2

o1o2 = get_A(np.diag([0.55, 1.45]), k).T

O = np.zeros((m,k))
O[:2,:2] = o1o2
O[2,1] = 1
O[3,0] = 1
O[4,0] = 1

# Returns O^*O
OstarO(O)

array([[ 2.55, -0.  ],
       [-0.  ,  2.45]])

# Not Clustered


In [3]:
m = 5
k = 2

o1o2 = get_A(np.diag([0.75, 1.25]), k).T
o3o4 = get_A(np.diag([0.8, 1.2]), k).T

O = np.zeros((m,k))
O[:2,:2] = o1o2 
O[2:4,:2] = o3o4
O[4,0] = 1

# Returns O^*O
print(OstarO(O))

# Looks nicer
O = np.array([
    [np.sqrt(3/8), -np.sqrt(5/8)],
    [np.sqrt(3/8),  np.sqrt(5/8)],
    [np.sqrt(2/5), -np.sqrt(3/5)],
    [np.sqrt(2/5),  np.sqrt(3/5)],
    [1,             0           ]        
])

# Returns O^*O
print(OstarO(O))

[[2.55 0.  ]
 [0.   2.45]]
[[2.55 0.  ]
 [0.   2.45]]


# Example of clusterization

In [4]:
import numpy as np
from matplotlib import pyplot as plt

from src.probability import Prior, Posterior
from src.forward import Heat
from src.observations import PointObservation, DiagObservation

%matplotlib

sig = 5e-2

N = 200
L = 1
time = 3e-2
alpha = 1.
gamma = -1.

## DST / DCT - discrete sine / cosine transform. 
## Corresponds to a homogeneous Dirichlet/ Neumann boundary condition.
## Clusterization also happens with Neumann boundary, I just chose to present 
## Dirichlet boundary
dst, dct = {}, {}
for transform in  ['dst']:#, 'dct']:
    delta = 0. if transform == 'dst' else 0.5
    fwd = Heat(N=N, L=L, transform=transform, alpha=alpha, time=time)
    pr = Prior(N=N, L=L, transform=transform, gamma=gamma, delta=delta)
    post = Posterior(fwd=fwd,
                     prior=pr,
                     sigSqr=sig**2,
                     L=L,
                     N=N,
                     transform=transform)
    dic = dst if transform == 'dst' else dct
    for m in range(2, 6):
        res = post.optimize(m=m, n_iterations=250)
        design = res['x']
        dic[m] = design
        #print(transform, m, design)


fs = 24
fig, ax = plt.subplots(figsize=(10,5))

for m, array in dst.items():
    colors = iter(list("brgkm"))

    vals = np.repeat(m, len(array))
    ax.scatter(array, vals, s=0)
    for i, val in enumerate(array):
        ax.annotate(str(i+1),
                    xy=(val, m),
                    ha='center', 
                    va='center',
                    fontsize=fs,
                    color=next(colors))
            

ax.set_xlabel('Measurement Location', fontsize=fs)
ax.set_ylabel('No. of Measurements', fontsize=fs)
ax.set_yticks(list(dst.keys()))
ax.set_xlim(0,1)
plt.tight_layout()

plt.savefig("arxiv/example.pdf")
plt.close()

Using matplotlib backend: QtAgg


# Eigenvectors and clusterization

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from src.probability import Prior, Posterior
from src.forward import Heat
from src.observations import PointObservation, DiagObservation

%matplotlib

sig = 5e-2
N = 200
L = 1
time = 3e-2
alpha = 1.
gamma = -1.
m = 4
transform = 'dst'
delta = 0. if transform == 'dst' else 0.5

fwd = Heat(N=N, L=L, transform=transform, alpha=alpha, time=time)
pr = Prior(N=N, L=L, transform=transform, gamma=gamma, delta=delta)
post = Posterior(fwd=fwd,
                 prior=pr,
                 sigSqr=sig**2,
                 L=L,
                 N=N,
                 transform=transform)

res = post.optimize(m=m, n_iterations=250)
print(res['x'])

In [15]:
plt.close('all')
fig = plt.figure(figsize=(8,4))
fs = 18

vals = np.zeros(m)
plt.scatter(res['x'], vals)
x = fwd.x
for i in range(4):
    ev = fwd.eigenvector(i)
    lam = np.sqrt(fwd.multiplier[i])
    plt.plot(x, ev * lam, label=i+1)
plt.plot([0,1], [0,0], ls='--', color='k')    
plt.xlabel(r"$x \in \Omega$", fontsize=fs)
plt.legend(title='eigenvector, weighted')
plt.tight_layout()
plt.savefig("arxiv/eigenvectors.pdf")
plt.close('all')